In [2]:
##Imports 
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
import numpy as np

In [31]:
"""Main code for bike data
    scrapping till the specs step""" 
# Note - might raise exceptions when a popup appears in between the scappring which leads to a stopage on a point, due to this navigation approach 
# Navigate back to the main bike page 
# driver.execute_script("window.history.go(-2)")" as "-2" defines the procedure to go 2 pages back in history point.
# We can bypass it by simply closing the pop-up then manually moving towards the main bikelist page it will retry automatically. 

#perfect one ,"ran successfully for the whole list "

def find_element_with_retry(driver, by, value, retries=3, delay=2):
    for _ in range(retries):
        try:
            element = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except (TimeoutException, StaleElementReferenceException) as e:
            print(f"Error finding element: {str(e)}. Retrying...")
            sleep(delay)
    raise Exception(f"Unable to find element {value} after {retries} retries.")

def select_dropdown_option(driver):
    dropdown_option_xpath = '//*[@id="rf01"]/header/div[1]/div/div/div[2]/div/div/form/div/div/div/ul/li'
    try:
        dropdown_options = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, dropdown_option_xpath))
        )
        if dropdown_options:
            dropdown_options[0].click()  # Clicking the first option
            return True
    except Exception as e:
        print(f"Error selecting dropdown option: {str(e)}")
        return False

def scrape_bike_details(search_query, driver):
    bike_details_list = []
    try:
        # Find the search text input element
        search_text_input = find_element_with_retry(driver, By.ID, 'cardekhosearchtext')

        # Input the search query into the search text input
        search_text_input.clear()
        search_text_input.send_keys(search_query)

        # Wait for a while to let the dropdown options appear
        sleep(1)

        # Re-find the dropdown options after waiting
        if not select_dropdown_option(driver):
            print(f"Failed to select dropdown option for query: {search_query}")
            return bike_details_list

        sleep(1)

        bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')

        for index in range(len(bike_list_element.find_elements(By.TAG_NAME, 'li'))):
            try:
                # Re-find the bike list element to avoid stale element reference
                bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')
                
                item = bike_list_element.find_elements(By.TAG_NAME, 'li')[index]
                x_element = item.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a')
                link = x_element.get_attribute('href')
                
                # Move to the element to ensure it's clickable
                ActionChains(driver).move_to_element(x_element).click().perform()

                # Additional scraping from the new page (e.g., bike name, price, etc.)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h1')))
                bike_name = driver.find_element(By.XPATH, '//h1').text

                # Fetch the price
                bike_price_element = driver.find_element(By.XPATH, '//div[@class="price"]')
                bike_price_text = bike_price_element.text.split('Rs.')[-1].split('*')[0].strip()

                # Check if the price contains a range
                if '-' in bike_price_text:
                    min_price, max_price = bike_price_text.split('-')
                    bike_price = f"{min_price.strip()} Lakh - {max_price.strip()}"
                else:
                    bike_price = f"{bike_price_text}"

                # Click on the "Specs" button to navigate to the specifications page
                specs_button = find_element_with_retry(driver, By.XPATH, '//a[contains(text(), "Specs")]')

                # Move to the element to ensure it's clickable
                ActionChains(driver).move_to_element(specs_button).click().perform()

                # Wait for the technical specifications section to be present
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'technicalSpecs')))

                # Find the technical specifications block
                tech_specs_block = driver.find_element(By.ID, 'technicalSpecs')

                # Find all h3 elements within the technical specifications block
                section_titles = tech_specs_block.find_elements(By.TAG_NAME, 'h3')

                for title_element in section_titles:
                    section_title = title_element.text
                    section_data = {}

                    # Find the corresponding table for each section
                    table = title_element.find_element(By.XPATH, './following-sibling::table[1]')

                    # Find all rows in the table
                    rows = table.find_elements(By.TAG_NAME, 'tr')

                    for row in rows:
                        columns = row.find_elements(By.TAG_NAME, 'td')
                        if len(columns) == 2:
                            feature = columns[0].text
                            value = columns[1].text
                            section_data[feature] = value

                    # Add the section data to the bike details list
                    bike_details_list.append({
                        'Search Query': search_query,
                        'Bike Name': bike_name,
                        'Price': bike_price,
                        'Section Title': section_title,
                        'Section Data': section_data
                    })

                # Navigate back to the main bike page
                driver.execute_script("window.history.go(-2)")
                
                # Wait for the bike list to load again
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'bikelist')))
                
            except NoSuchElementException as ne:
                print(f"No <h3> tag found for item {search_query}. Skipping...")
                continue
            except Exception as e:
                print(f"An error occurred while processing item {search_query}: {str(e)}")
                continue

    except NoSuchElementException as ne:
        print(f"No element found: {ne}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    return bike_details_list

try:
    bike_details_list = []

    with webdriver.Chrome() as driver:
        driver.get('https://www.bikedekho.com')
        search_queries = [
            'ola bike'
        ]
        for query in search_queries:
            result = scrape_bike_details(query, driver)
            bike_details_list.extend(result)

    # Convert the list of dictionaries to a pandas DataFrame
    dfc = pd.DataFrame(bike_details_list)

    # Save the DataFrame to a CSV file
    # dfz.to_csv('bike_details.csv', index=False)

except Exception as e:
    print(f"An error occurred: {str(e)}")


In [32]:
dfc

,Search Query,Bike Name,Price,Section Title,Section Data
0,ola bike,Ola S1 Pro,1.30 Lakh,Engine and Transmission,"{'Continuous Power': '5.5 kw', 'Motor Power': ..."
1,ola bike,Ola S1 Pro,1.30 Lakh,Features,"{'Instrument Console': 'Digital', 'Bluetooth C..."
2,ola bike,Ola S1 Pro,1.30 Lakh,Features and Safety,"{'Braking Type': 'Combine Braking System', 'Ch..."
3,ola bike,Ola S1 Pro,1.30 Lakh,Chassis and Suspension,{'Body Type': 'Electric Bikes'}
4,ola bike,Ola S1 Pro,1.30 Lakh,Dimensions and Capacity,"{'Width': '712 mm', 'Length': '1859 mm', 'Heig..."
5,ola bike,Ola S1 Pro,1.30 Lakh,Electricals,"{'Headlight': 'LED', 'Tail Light': 'LED', 'Tur..."
6,ola bike,Ola S1 Pro,1.30 Lakh,Tyres and Brakes,"{'Front Brake Diameter': '220 mm', 'Rear Brake..."
7,ola bike,Ola S1 Pro,1.30 Lakh,Performance,"{'Scooter Speed': 'high', '0-40 Kmph (sec)': '..."
8,ola bike,Ola S1 Pro,1.30 Lakh,Motor & Battery,"{'Motor Type': 'Mid Drive IPM', 'Continious Po..."
9,ola bike,Ola S1 Pro,1.30 Lakh,Range,{'Claimed Range': '195 km/charge'}


In [20]:
# 'Bajaj bike', 'TVS bike', 'Royal Enfield bike',
#             'Honda bike', 'Yamaha bike', 'Suzuki bike', 'KTM bike',
#             'Jawa bike', 'Benelli bike', 'Harley bike', 'Triumph bike',
#             'Ducati bike', 'BMW bike', 'Kawasaki bike', 'Husqvarna bike',
#             'Revolt bike', 'Ather bike'
# # dfz['Bike Name'].nunique()

# # print(dfz.isnull().sum())

# # dfz.to_csv('bike_specZ.csv', index=False)

Search Query     0
Bike Name        0
Price            0
Section Title    0
Section Data     0
dtype: int64


In [ ]:
# ['Yamaha MT 15 V2', 'Yamaha R15 V4', 'Yamaha R15S',
#        'Yamaha FZS-FI V3', 'Yamaha RayZR 125 Fi Hybrid',
#        'Yamaha Aerox 155', 'Yamaha FZ-FI V3',
#        'Yamaha Fascino 125 Fi Hybrid', 'Yamaha FZS-FI V4', 'Yamaha FZ X',
#        'Yamaha R3', 'Yamaha MT-03', 'Yamaha FZ 25', 'Yamaha FZS 25'].sort

In [ ]:
#working fine for specz extraction using specs page link

# Initialize the browser
driver = webdriver.Chrome()

# Navigate to the URL
driver.get('https://www.bikedekho.com/ola-electric/s1-pro/specifications')

try:
    # Wait for the technical specifications section to be present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'technicalSpecs')))

    # Find the technical specifications block
    tech_specs_block = driver.find_element(By.ID, 'technicalSpecs')

    # Find all h3 elements within the technical specifications block
    section_titles = tech_specs_block.find_elements(By.TAG_NAME, 'h3')

    for title_element in section_titles:
        section_title = title_element.text
        print(f"=== {section_title} ===")
        
        # Find the corresponding table for each section
        table = title_element.find_element(By.XPATH, './following-sibling::table[1]')
        
        # Find all rows in the table
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, 'td')
            if len(columns) == 2:
                feature = columns[0].text
                value = columns[1].text
                print(f"{feature}: {value}")
        
        print("\n")  # Separate sections with a newline

finally:
    # Close the browser
    driver.quit()


In [ ]:
#working fine for bike main page data fetching 

def find_element_with_retry(driver, by, value, retries=3, delay=2):
    for _ in range(retries):
        try:
            element = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except (TimeoutException, StaleElementReferenceException) as e:
            print(f"Error finding element: {str(e)}. Retrying...")
            sleep(delay)
    raise Exception(f"Unable to find element {value} after {retries} retries.")

try:
    # Initialize the browser
    driver = webdriver.Chrome()

    # Navigate to the URL
    driver.get('https://www.bikedekho.com')

    # List of search queries
    search_queries = [
        'ola bike', 'Bajaj bike', 'TVS bike', 'Royal Enfield bike',
        'Honda bike', 'Yamaha bike', 'Suzuki bike', 'KTM bike',
        'Jawa bike', 'Benelli bike', 'Harley bike', 'Triumph bike',
        'Ducati bike', 'BMW bike', 'Kawasaki bike', 'Husqvarna bike',
        'Revolt bike', 'Ather bike'
    ]

    bike_details = []

    for search_query in search_queries:
        # Find the search text input element
        search_text_input = find_element_with_retry(driver, By.ID, 'cardekhosearchtext')

        # Input the search query into the search text input
        search_text_input.clear()
        search_text_input.send_keys(search_query)

        # Wait for a while to let the dropdown options appear
        sleep(2)

        # Re-find the dropdown options after waiting
        dropdown_option_xpath = '//*[@id="rf01"]/header/div[1]/div/div/div[2]/div/div/form/div/div/div/ul/li'
        dropdown_options = find_element_with_retry(driver, By.XPATH, dropdown_option_xpath, retries=5, delay=2)

        # Select the first option from the dropdown if available
        if dropdown_options:
            first_option = dropdown_options
            driver.execute_script("arguments[0].scrollIntoView();", first_option)
            first_option.click()

        sleep(2)
        
        bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')

        for index in range(len(bike_list_element.find_elements(By.TAG_NAME, 'li'))):
            try:
                # Re-find the bike list element to avoid stale element reference
                bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')
                
                item = bike_list_element.find_elements(By.TAG_NAME, 'li')[index]
                x_element = item.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a')
                link = x_element.get_attribute('href')
                bike_details.append(link)
                
                # Click on the link to navigate to the bike details page
                x_element.click()

                # Additional scraping from the new page (e.g., bike name, price, etc.)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h1')))
                bike_name = driver.find_element(By.XPATH, '//h1').text
                bike_price = driver.find_element(By.XPATH, '//span[@class="price"]').text
                
                print(f"Bike Name: {bike_name}, Price: {bike_price}")

                # Navigate back to the previous page
                driver.execute_script("window.history.go(-1)")
                
                # Wait for the bike list to load again
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'bikelist')))
                
            except NoSuchElementException:
                print(f"No <h3> tag found for item {search_query}. Skipping...")
                continue
            except Exception as e:
                print(f"An error occurred while processing item {search_query}: {str(e)}")
                continue

    bikes_df_basicss = pd.DataFrame(bike_details, columns=['Bike URLs'])
    # print(bikes_df_basicss)

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    # Close the browser
    if driver:
        driver.quit()
